# Proje Özeti

## Dataset Hakkında
- Bu çalışmada kullanılan veri seti: [Brain Tumor MRI Dataset](https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset/data)  
- Veri seti **4 sınıftan oluşmaktadır**:  
  - **Glioma**  
  - **Meningioma**  
  - **Pituitary**  
  - **No Tumor**  
- Toplamda **7,023 MRI görüntüsü** içermektedir. Görseller farklı boyutlarda olup gri/renkli varyasyonlar barındırmaktadır.  
- Amaç: MRI görüntülerinde **beyin tümörünün varlığını ve tipini sınıflandırmak**, böylece medikal görüntüleme alanında otomatik bir yardımcı sistem geliştirmektir.  

***
## <span style='border-left: 4px solid #0000FF; padding-left: 10px;'> İçerikler <b>

1. [`Dataset Hazırlığı`](#data)
2. [`Sınıf Dağılımı Analizi ve Görselleştirme`](#imports)
3. [`CNN Modeli Kurulumu ve Derleme`](#import_data)
4. [`Model Performans Değerlendirmesi`](#vis)

Author: [Dilşah Bahçeci](https://www.kaggle.com/dilahbaheci)

# 1. Dataset Hazırlığı
## 1.1. Brain Tumor MRI Dataset Hazırlığı

Bu notebook'ta MRI beyin tümörü veri seti (glioma, meningioma, pituitary, notumor) üzerinde 
**dosya yolları** ve **etiketleri** toplayıp pandas DataFrame formatına dönüştürüyoruz.  

- `get_data_labels()` fonksiyonu klasör yapısını dolaşarak resim dosyalarının yollarını ve etiketlerini çıkarır.  
- Eğitim (`Training`) ve test (`Testing`) klasörlerinden veriler ayrı ayrı alınır.  
- Sonuç olarak `train_df` ve `test_df` DataFrame’leri oluşturulur.  
  - `Path`: Resim dosyasının tam yolu  
  - `Label`: Resmin ait olduğu sınıf  

Böylece veri seti daha sonra **model eğitimi** ve **analiz** için hazır hale gelir.


In [ ]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

def get_data_labels(directory):
    paths = []
    labels = []
    classes = os.listdir(directory)  # klasör isimleri: glioma, meningioma, pituitary, notumor
    
    for label in classes:
        class_dir = os.path.join(directory, label)
        for file in os.listdir(class_dir):
            file_path = os.path.join(class_dir, file)
            if file_path.endswith(('.jpg', '.png', '.jpeg')):  # sadece resimleri al
                paths.append(file_path)
                labels.append(label)
    
    return paths, labels


train_paths, train_labels = get_data_labels('/kaggle/input/brain-tumor-mri-dataset/Training')
test_paths, test_labels = get_data_labels('/kaggle/input/brain-tumor-mri-dataset/Testing')

train_df = pd.DataFrame({"Path": train_paths,"Label": train_labels})
test_df = pd.DataFrame({"Path": test_paths,"Label": test_labels})

train_df, test_df.head()



## 1.2. Etiketlerin Sayısallaştırılması (Label Encoding)

Bu adımda, sınıf etiketlerini (glioma, meningioma, pituitary, notumor) 
**string** formatından **sayısal** (0, 1, 2, 3) formata dönüştürüyoruz.  

- `LabelEncoder` sınıfı ile string etiketler integer değerlere çevrilir.  
- Eğitim (`train_labels`) üzerinde `fit_transform()` uygulanır, böylece sınıflar öğrenilir.  
- Test (`test_labels`) üzerinde `transform()` uygulanır, aynı mapping kullanılır.  
- Daha sonra `train_df` ve `test_df` DataFrame’leri oluşturularak:  
  - `Path`: Resim dosyasının yolu  
  - `Label`: Sayısal sınıf etiketi  
  saklanır.  

Son olarak `head()` fonksiyonu ile ilk birkaç satır kontrol edilir.


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Label encode (string -> integer)
le = LabelEncoder()
train_labels_int = le.fit_transform(train_labels)  # 0,1,2,3
test_labels_int = le.transform(test_labels)

# DataFrame oluştur
train_df = pd.DataFrame({"Path": train_paths, "Label": train_labels_int})
test_df = pd.DataFrame({"Path": test_paths, "Label": test_labels_int})

# Kontrol
print(train_df.head())
print(test_df.head())


'glioma'      → 0
'meningioma'  → 1
'notumor'     → 2
'pituitary'   → 3


## 1.3. Görüntülerin NumPy Dizisine Çevrilmesi

Bu adımda, DataFrame içerisindeki resim yolları kullanılarak resimler yüklenir, 
ön işleme tabi tutulur ve model için hazır hale getirilir.  

- `load_img`: Resimleri belirtilen hedef boyuta (`128x128`) ve gri tonlamalı (grayscale) olarak yükler.  
- `img_to_array`: Yüklenen resmi NumPy dizisine çevirir.  
- Normalizasyon: Piksel değerleri **0-255** aralığından **0-1** aralığına dönüştürülür.  
- Etiketler (`Label` sütunu) `class_names` listesine göre integer değerine çevrilir.  
- Sonuç:  
  - `X_train`, `X_test`: Resim verileri (NumPy dizisi)  
  - `y_train`, `y_test`: Sayısal sınıf etiketleri  

Son olarak `X_train.shape` ile eğitim setinin boyutu kontrol edilir.


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

IMG_SIZE = (128, 128)  # model input boyutu

def df_to_arrays(df, class_names):
    X = []
    y = []
    for path, label in zip(df['Path'], df['Label']):
        img = load_img(path, target_size=IMG_SIZE, color_mode='grayscale')   # resimleri hedef boyuta getir
        img_array = img_to_array(img) / 255.0        # normalize [0,1]
        X.append(img_array)
        y.append(class_names.index(label))           # string label → integer
    return np.array(X), np.array(y)


CLASS_TYPES = [3, 2, 1, 0]

X_train, y_train = df_to_arrays(train_df, CLASS_TYPES)
X_test, y_test = df_to_arrays(test_df, CLASS_TYPES)

print(X_train.shape)

# 2. Sınıf Dağılımı Analizi ve Görselleştirme
## 2.1. Örnek Görselleri Gösterme

- `LabelEncoder` ile string etiketler sayısal değerlere (`0, 1, 2, 3`) dönüştürüldü.
- Sınıf isimleri `class_names` değişkenine alındı (`glioma`, `meningioma`, `notumor`, `pituitary`).
- `show_sample_images` fonksiyonu kullanılarak her sınıftan 3 rastgele görsel seçildi ve gösterildi:
  - Önce **train seti** için örnekler görüntülendi.
  - Ardından **test seti** için örnekler gösterildi.


In [ ]:
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
import random

def show_sample_images(df, class_names, num_samples=3, img_size=(128,128)):
    for cls_idx, cls_name in enumerate(class_names):
        class_images = df[df['Label'] == cls_idx]['Path'].tolist()
        if len(class_images) == 0:
            continue
        sample_paths = random.sample(class_images, min(num_samples, len(class_images)))
        
        plt.figure(figsize=(15,5))
        for i, img_path in enumerate(sample_paths):
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, img_size)
            
            plt.subplot(1, num_samples, i+1)
            plt.imshow(img, cmap='gray')
            plt.axis('off')
            plt.title(f"{cls_name}")
        plt.show()

# Örnek: train_df['Label'] string ise önce encode et
le = LabelEncoder()
train_df['Label'] = le.fit_transform(train_df['Label'])
test_df['Label'] = le.transform(test_df['Label'])

# Sınıf isimlerini al
class_names = le.classes_   # ['glioma','meningioma','notumor','pituitary']

show_sample_images(train_df, class_names, num_samples=3)
show_sample_images(test_df, class_names, num_samples=3)


## 2.2. Sınıf Bazında Train/Test Dağılımı Analizi(Pie Chart)

Bu adımda her bir sınıf için (glioma, meningioma, notumor, pituitary) 
**eğitim ve test veri dağılımları** görselleştirilir.  

- `class_names`: Etiket isimleri (`['glioma','meningioma','notumor','pituitary']`).  
- `class_indices`: Her sınıfa karşılık gelen integer değerler (`[0,1,2,3]`).  
- Döngü ile her sınıf için:  
  - Eğitim setindeki örnek sayısı (`train_count`).  
  - Test setindeki örnek sayısı (`test_count`).  
  - Toplam örnek sayısı (`total`).  
- **Pie chart**: Eğitim ve test oranlarını yüzde ve adet bazında gösterir.  

Sonuç: Her sınıf için ayrı ayrı `Train/Test` dağılımlarını görselleştire


In [ ]:
import matplotlib.pyplot as plt

# Sınıf isimleri ve integer karşılıkları
class_names = le.classes_      # ['glioma','meningioma','notumor','pituitary']
class_indices = range(len(class_names))  # [0,1,2,3]

# Her sınıf için pie chart
for cls_idx, cls_name in zip(class_indices, class_names):
    train_count = (train_df['Label'] == cls_idx).sum()
    test_count = (test_df['Label'] == cls_idx).sum()
    total = train_count + test_count
    
    if total == 0:
        print(f"{cls_name} sınıfında veri yok, atlanıyor.")
        continue
    
    sizes = [train_count, test_count]
    labels = [f'Train ({train_count} / {total}, {train_count/total*100:.1f}%)',
              f'Test ({test_count} / {total}, {test_count/total*100:.1f}%)']
    
    plt.figure(figsize=(5,5))
    plt.pie(sizes, labels=labels, autopct='', startangle=90, colors=['skyblue', 'lightgreen'])
    plt.title(f"{cls_name} sınıfı - Train/Test dağılımı")
    plt.show()


## 2.3. Eğitim ve Test Seti Sınıf Dağılımı Analizi(Pie Chart)

Bu adımda eğitim ve test veri setlerindeki **sınıf yüzdeleri** 
pasta grafikleri ile görselleştirilir.  

- `train_counts` / `test_counts`: Her sınıftaki örnek sayıları.  
- `train_percent` / `test_percent`: Her sınıfın toplam veri içindeki yüzdesi.  
- `class_names`: Etiket isimleri (`glioma, meningioma, notumor, pituitary`).  
- İki ayrı subplot oluşturulur:  
  - **Sol grafik**: Eğitim seti sınıf dağılımı  
  - **Sağ grafik**: Test seti sınıf dağılımı  

Sonuç: Eğitim ve test setlerindeki sınıf dengesizlikleri kolayca karşılaştırılabilir.


In [ ]:
import matplotlib.pyplot as plt

# Eğitim seti yüzdeleri
train_counts = train_df['Label'].value_counts().sort_index()
train_percent = (train_counts / len(train_df) * 100).round(2)
class_names = le.classes_

# Test seti yüzdeleri
test_counts = test_df['Label'].value_counts().sort_index()
test_percent = (test_counts / len(test_df) * 100).round(2)

# Pasta grafikleri
plt.figure(figsize=(12,6))

# Train set
plt.subplot(1,2,1)
plt.pie(train_percent, labels=class_names, autopct='%1.1f%%', startangle=90, colors=['skyblue','salmon','lightgreen','orange'])
plt.title('Train Set Sınıf Dağılımı')

# Test set
plt.subplot(1,2,2)
plt.pie(test_percent, labels=class_names, autopct='%1.1f%%', startangle=90, colors=['skyblue','salmon','lightgreen','orange'])
plt.title('Test Set Sınıf Dağılımı')

plt.show()


- Soldaki pie chart, **Train setindeki** sınıf dağılımını gösteriyor.  
  - `notumor` sınıfı en yüksek oran (%27.9) ile öne çıkıyor.  
  - Diğer sınıflar (`glioma`, `meningioma`, `pituitary`) yaklaşık olarak %23-25 oranında.  

- Sağdaki pie chart, **Test setindeki** sınıf dağılımını gösteriyor.  
  - `notumor` sınıfı yine en yüksek oran (%30.9).  
  - Diğer sınıflar (%22-23 civarı) neredeyse eşit dağılım göstermekte.  

- **Genel yorum:** Hem train hem test setlerinde sınıflar nispeten dengeli dağılmış, `notumor` sınıfı biraz daha fazla temsil edilmiş. Bu, sınıf dengesizliğinin minimal olduğunu gösteriyor ve model eğitimi için olumlu bir durum.  


# 3.CNN Modeli Kurulumu ve Derleme

Bu adımda beyin tümörü sınıflandırması için bir **Convolutional Neural Network (CNN)** modeli tanımlanır.  

### Model Mimarisi
- **Girdi katmanı**: `(128,128,1)` boyutunda gri ölçekli resimler.  
- **Konvolüsyon + MaxPooling katmanları**:  
  - Conv2D(32 filtre) + MaxPooling2D  
  - Conv2D(64 filtre) + MaxPooling2D  
  - Conv2D(128 filtre) + MaxPooling2D  
- **Flatten**: Çok boyutlu çıktıyı düzleştirir.  
- **Dense (Tam bağlı katman)**: 128 nöron + ReLU aktivasyonu.  
- **Dropout (0.5)**: Overfitting’i azaltmak için rastgele nöronları kapatır.  
- **Çıkış katmanı**: `num_classes` nöron, Softmax aktivasyonu (çok sınıflı sınıflandırma).  

### Öğrenme Oranı Düzenleme
- `ReduceLROnPlateau`: Valide kaybı (`val_loss`) iyileşmediğinde öğrenme oranını yarıya indirir.  

### Derleme
- **Optimizer**: Adam (learning rate = 0.001)  
- **Loss**: `sparse_categorical_crossentropy` (etiketler integer olduğu için)  
- **Metrik**: Accuracy  

Son olarak `model.summary()` ile modelin katman yapısı ve parametre sayıları görüntülenir.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Sınıf sayısı
num_classes = len(CLASS_TYPES)

# Modeli oluştur
model = Sequential([
    # 1. Conv + Pool
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,1)),
    MaxPooling2D((2,2)),

    # 2. Conv + Pool
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    # 3. Conv + Pool
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    # Flatten ve Dense katmanları
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # overfitting'i azaltmak için
    Dense(num_classes, activation='softmax')  # sınıf sayısı kadar çıktı
])

from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',   # val_accuracy de seçebilirsin
    factor=0.5,           # LR’i yarıya indir
    patience=2,           # 2 epoch boyunca gelişme yoksa uygula
    min_lr=1e-6           # minimum LR sınırı
)

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)



# Model özeti
model.summary()


## 3.1. Eğitim Setini Eğitim ve Validation Olarak Bölme

Bu adımda, mevcut **eğitim verileri** (`X_train`, `y_train`) içerisinden %10’u 
**validation seti** olarak ayrılır.  

- `test_size=0.1`: Eğitim setinin %10’u validation setine ayrılır.  
- `random_state=42`: Aynı bölünmenin tekrar üretilebilmesi için sabit değer.  
- `stratify=y_train`: Eğitim ve validation setinde sınıf dağılımlarının korunmasını sağlar.  

Sonuç olarak:  
- `X_train_new`, `y_train_new`: Yeni eğitim seti (%90)  
- `X_val`, `y_val`: Validation seti (%10)  

Son satırda setlerin boyutları ekrana yazdırılarak kontrol edilir.


In [ ]:
from sklearn.model_selection import train_test_split

# Train setini %90 eğitim, %10 validation olarak ayır
X_train_new, X_val, y_train_new, y_val = train_test_split(
    X_train, y_train, 
    test_size=0.1,      # %10 validation
    random_state=42,    # tekrar üretilebilirlik için sabit sayı
    stratify=y_train    # sınıf dağılımını korumak için
)

print("Yeni eğitim seti:", X_train_new.shape, y_train_new.shape)
print("Validation seti:", X_val.shape, y_val.shape)


## 3.2. Modelin Eğitimi

Bu adımda CNN modeli eğitim verileri üzerinde eğitilir.  

- **`model.fit()` parametreleri**:  
  - `X_train, y_train`: Eğitim verileri.  
  - `validation_data=(X_test, y_test)`: Test seti doğrulama için kullanılır. (Alternatif olarak validation seti kullanılabilir.)  
  - `epochs=20`: Eğitim 20 epoch boyunca devam eder.  
  - `callbacks=[reduce_lr]`: `ReduceLROnPlateau` callback’i, doğrulama kaybı iyileşmezse öğrenme oranını düşürür.  

Sonuç olarak, eğitim süreci `history` değişkeninde saklanır. Bu değişken daha sonra loss/accuracy grafiklerini çizmek için kullanılabilir.


In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    callbacks=[reduce_lr]
)

In [ ]:
from tensorflow.keras.models import load_model

# Eğer modeli kaydetmediysen önce kaydet:
model.save("my_model.h5")

# Test setinde performansı ölç
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")


## 4. Model Performans Değerlendirmesi
# 4.1. Sınıflandırma Raporu

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

# --- 1. Tahminler ---
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)   # en yüksek olasılığı seç
y_true = y_test                            # zaten label olarak var (1D)

# --- 2. Classification Report ---
print("Classification Report:")
print(classification_report(y_true, y_pred))

Modelin performans metrikleri oldukça yüksek seviyede. Toplam **1311 örnek** üzerinde test edilmiş ve **%98 doğruluk (accuracy)** elde edilmiş. Hem **macro avg** hem de **weighted avg** değerleri %98 civarında olduğundan, modelin farklı sınıflar arasında dengeli bir başarı gösterdiği söylenebilir.

**Sınıf Bazlı Sonuçlar**
- **Sınıf 0**  
  - Precision: **0.99**, Recall: **1.00**, F1: **1.00**  
  - Bu sınıf neredeyse kusursuz sınıflandırılmış. Hiç hata yapılmamış denebilir.  

- **Sınıf 1**  
  - Precision: **0.99**, Recall: **1.00**, F1: **0.99**  
  - Çok yüksek başarı. Birkaç küçük hata dışında mükemmele yakın.  

- **Sınıf 2**  
  - Precision: **0.96**, Recall: **0.95**, F1: **0.96**  
  - Performans diğer sınıflara göre biraz daha düşük, özellikle recall tarafında. Bu, bazı **sınıf 2 örneklerinin yanlış sınıflandırıldığını** gösteriyor.  

- **Sınıf 3**  
  - Precision: **0.97**, Recall: **0.96**, F1: **0.97**  
  - Gayet başarılı, yalnızca sınıf 2’ye benzer şekilde birkaç hata mevcut.  

**Genel Yorum**
- Model **yüksek doğruluk oranı** ve **tüm sınıflar için dengeli sonuçlar** veriyor.  
- **Sınıf 2**'de recall değeri diğer sınıflara göre daha düşük olduğundan, model bu sınıfı ayırt etmekte biraz zorlanıyor olabilir.  
- Genel olarak **üretim için kullanılabilir seviyede güçlü bir model** olduğu söylenebilir.  

👉 İyileştirme için: sınıf 2’ye ait daha fazla veri eklenebilir veya veri dengesizliği varsa **data augmentation / class weighting** teknikleri denenebilir.


## 4.2.Karışıklık Matriksi

In [ ]:
# --- 3. Confusion Matrix ---
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)

plt.xlabel("Tahmin Edilen")   # Predicted
plt.ylabel("Gerçek")          # True
plt.title("Karışıklık Matrisi") # Confusion Matrix
plt.tight_layout()
plt.show()



In [ ]:
# --- 4. Learning Curves ---
# Accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Accuracy per Epoch")
plt.show()


## 4.3.Eğitim ve Doğrulama Doğruluğu (Accuracy) Grafiği

Grafikte **epoch** sayısına karşılık eğitim (mavi) ve doğrulama (turuncu) doğruluklarının değişimi gösterilmektedir.  

- Eğitim doğruluğu **hızlı bir şekilde artmış** ve 20 epoch sonunda **%99 seviyelerine ulaşmıştır**.  
- Doğrulama doğruluğu da benzer şekilde artmış, **%97–98 seviyelerine ulaşarak** eğitim doğruluğuna oldukça yakın kalmıştır.  

**Yorum**
- Model, hem eğitim hem de doğrulama setinde yüksek doğruluk elde etmiştir.  
- **Overfitting (aşırı öğrenme) gözlenmemektedir**, çünkü eğitim ve doğrulama doğrulukları arasında büyük bir fark yoktur.  
- Eğitimin başlarında (ilk 3-4 epoch) doğrulama eğrisi eğitim eğrisine yakın seyretmiş, sonrasında da paralel bir şekilde artmaya devam etmiştir.  
- Bu sonuçlar, modelin **genelleme kabiliyetinin güçlü** olduğunu göstermektedir.  

**Sonuç**
Model başarılı bir şekilde öğrenmiş ve doğrulama setinde yüksek performans sergilemiştir. Daha fazla epoch denenmesine gerek olmayabilir; mevcut haliyle model üretim için oldukça uygun görünmektedir.


In [ ]:

# Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.title("Loss per Epoch")
plt.show()

## 4.4. Eğitim ve Doğrulama Kayıp (Loss) Grafiği

Grafikte **epoch** sayısına karşılık eğitim (mavi) ve doğrulama (turuncu) kayıplarının değişimi gösterilmektedir.  

- Eğitim kaybı (train loss) düzenli olarak azalmış ve yaklaşık **0.01 seviyelerine kadar düşmüştür**.  
- Doğrulama kaybı (validation loss) da genel olarak azalmış, **0.1 civarında sabitlenmiştir**.  
- Eğitim ve doğrulama kayıpları arasındaki fark epoch ilerledikçe biraz artmıştır.  

**Overfitting Analizi**
- Eğer doğrulama kaybı, eğitim kaybına kıyasla yükselip dalgalanma gösterseydi **overfitting** işareti olurdu.  
- Bu grafikte doğrulama kaybı, eğitim kaybından biraz yüksek kalmış olsa da **stabil bir seviyede** devam etmektedir.  
- Dolayısıyla, **belirgin bir overfitting gözlenmemektedir**. Küçük farklar normaldir ve modelin genelleme yapabildiğini gösterir.  

**Sonuç**
- Model, eğitim verisini iyi öğrenmiş ve doğrulama verisinde de düşük kayıp seviyesine ulaşmıştır.  
- Eğitim ve doğrulama kayıpları arasındaki fark düşük olduğu için modelin **genelleme başarısı yüksektir**.  
- Bu haliyle model, üretim ortamında kullanılabilir durumda görünmektedir.
